In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
f=0
for dirname, _, filenames in os.walk('/kaggle/input/mycroppedimages'):
    print(dirname)
    for filename in filenames:
        if dirname=='/kaggle/input/mycroppedimages/myCroppedImages':
           f+=1
        
print('number of files is : ',f)
del dirname, filenames, f, filename
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mycroppedimages
/kaggle/input/mycroppedimages/myCroppedImages
/kaggle/input/mycroppedimages/myCroppedImages/IntermediaryImages
number of files is :  55092


# Install necessary packages  

In [2]:
!pip install opencv-python

In [3]:
!python3 -m ensurepip


Looking in links: /tmp/tmpbvlu56lr


In [4]:
!pip install paddleocr --user

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 544.7/544.7 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.9/161.9 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 73.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 100.6 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114248 sha256=9403efce124985212b8dd634167aeb5321d5c4cb90e4d5babd29da07e373aad7
  Stored in directory: /root/.cache/pip/wheels/19/39/2f/2d3cadc408a8804103f1c34ddd4b9f6a93497b11fa96fe738e
Successfully built fire
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-locati

In [5]:
!pip install "paddleocr>=2.0.1"

In [6]:
!pip install paddlepaddle
!pip install paddlepaddle-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 758.9/758.9 MB 2.1 MB/s eta 0:00:00


# Define functions used to process all images

In [7]:

#Used to reset the working directory
def DeleteAllWorkingDir(workDir = '/kaggle/working/'):

    for item in os.listdir(workDir):
        isFile = os.path.join(workDir, item) #Concatenate filename in working directory with working directory path  
        if os.path.isfile(isFile):
            os.remove(isFile)  # Delete files directly in working directory
        elif os.path.isdir(isFile):
            shutil.rmtree(isFile)  # Delete directory and files in that directory

    print(f"Tout le contenu de {workDir} a été supprimé.")

In [8]:
#Main function to process a cropped image


# Désactiver les logs de PaddleOCR
#import logging
#logging.getLogger("ppocr").setLevel(logging.ERROR)


def paddleOCRWhiteRegions (myImagePath, myImageName, destIntermedDir, whiteThresh=0.45):
   
    srcImg=myImagePath+myImageName
   
    Img=cv2.imread(srcImg, cv2.IMREAD_COLOR)

    Img=cv2.resize(Img,(0,0), fx=2.0, fy=2.0, interpolation=cv2.INTER_NEAREST) 
   
    grayImg=cv2.cvtColor(Img, cv2.COLOR_BGR2GRAY)
   
    threshImage=cv2.adaptiveThreshold(grayImg, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 15, 5)
    
    ocrPath=destIntermedDir+"ocr_image.png"
    cv2.imwrite(ocrPath, Img)
    
    
    myDictInfos = {
        'ImageName': myImageName,
        'Frame_Number': int(myImageName.split('_')[-1].split('.')[0]),
        'Detected_Text': {
            'Text_Box_(x,y,w,h)': [],
            'Text': [],
            'Confidence': [],
            'White_Ratio': [],
            'FilePath': []
        }
    }

    
    results=ocr.ocr(ocrPath, cls=True)
    
    if not results[0]:
        print("No text detected")
        return myDictInfos
    for line in results[0]:
        coord, (txt, confidence) = line
        xMin=int(min([point[0] for point in coord]))
        xMax=int(max([point[0] for point in coord]))
        yMin=int(min([point[1] for point in coord]))
        yMax=int(max([point[1] for point in coord]))
        region=threshImage[yMin:yMax, xMin:xMax]
        h,w=yMax-yMin, xMax-xMin
        totalPixels = region.size
        whitePixels = cv2.countNonZero(region)
        whiteRatio = whitePixels / float(totalPixels)
        
        if whiteRatio>=whiteThresh and (xMin>300 or yMin>20):
                myRegionImagePath=destIntermedDir+'Img_{}_Region_{}_WRatio_{}_txt_{}_conf_{}.png'.format(myDictInfos['Frame_Number'],(xMin,yMin,w,h), np.round(whiteRatio,2), txt, np.round(confidence,3))
                myDictInfos['Detected_Text']['Text_Box_(x,y,w,h)'].append((xMin,yMin,w,h))
                
                myDictInfos['Detected_Text']['Text'].append(txt)
                
                myDictInfos['Detected_Text']['Confidence'].append(confidence)
                
                myDictInfos['Detected_Text']['White_Ratio'].append(whiteRatio)
                
                myDictInfos['Detected_Text']['FilePath'].append(myRegionImagePath)
                
                cv2.imwrite(myRegionImagePath,region)
                
    return myDictInfos




In [9]:
#make a folder to store OCR'ed sub-images
def makeFolder(dirPass, dirName):
    try:
        if not os.path.isabs(dirPass):
            raise ValueError(f"Le chemin n'est pas absolu {dirPass}")
        if not os.path.exists(dirPass+dirName):
            os.mkdir(dirPass+dirName, mode=0o700)   
            return True
        else:
            return False
    except ValueError as e:
        print(f"Erreur : {e}")
        return None

In [10]:
#Function to store a new line in a json file in order to store all the metadata from paddleOCRwhiteRegion() function
def append_to_json_lines(file_path, new_data):
    if not os.path.exists(file_path):
        with open(file_path, 'w') as f:
            print(f"file '{file_path}' created")
            f.write(json.dumps(new_data) + ', \n')
    else:
        # Ouvrir le fichier en mode append
        with open(file_path, 'a') as f:
            f.write(json.dumps(new_data) + ', \n')

In [11]:
#%%
#myGlobalDir='/kaggle/working/OCRImages/'
#myImagePath='/kaggle/input/mycroppedimages/myCroppedImages/'
#n=1    

#Function to process one image and store it in json file number n 
def processOneImageOCR(myImagePath, myImageName, myGlobalDir, n):
    newDir=myImageName.removesuffix('.jpg').split('_')[-1]
    isCreated=makeFolder(myGlobalDir, newDir)
    if isCreated:
        oneImageOCR=paddleOCRWhiteRegions (myImagePath, myImageName, myGlobalDir+newDir+'/', whiteThresh=0.45)
        
        append_to_json_lines(f'/kaggle/working/ocr_results_{n}.json', oneImageOCR)
    else:
        print('Creation of folder resulted in an error : ', isCreated, newDir)

# Configure Paddle to work with GPU and launch main loop

In [12]:
from paddleocr import PaddleOCR, draw_ocr
import paddle
import cv2

import numpy as np
import pandas as pd

import os 
import json
import shutil

gpu_available  = paddle.device.is_compiled_with_cuda()
print("GPU available:", gpu_available)

os.environ["OMP_NUM_THREADS"] = "8"

#Call the OCR model and download it
ocr=PaddleOCR(use_angle_cls=True, lang='en', use_gpu="True", cpu_threads=8) 

GPU available: True
download https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_det_infer.tar to /root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer/en_PP-OCRv3_det_infer.tar


100%|██████████| 3910/3910 [00:17<00:00, 229.74it/s] 


download https://paddleocr.bj.bcebos.com/PP-OCRv4/english/en_PP-OCRv4_rec_infer.tar to /root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer/en_PP-OCRv4_rec_infer.tar


100%|██████████| 10000/10000 [00:17<00:00, 583.06it/s]


download https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar to /root/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer/ch_ppocr_mobile_v2.0_cls_infer.tar


100%|██████████| 2138/2138 [00:14<00:00, 150.89it/s]

[2025/03/02 13:07:46] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu='True', use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_

[2025/03/02 13:07:47] ppocr WARNING: The first GPU is used for inference by default, GPU ID: 0
[2025/03/02 13:07:48] ppocr WARNING: The first GPU is used for inference by default, GPU ID: 0


In [13]:
#Start with a fresh working directory
DeleteAllWorkingDir(workDir = '/kaggle/working/')

Tout le contenu de /kaggle/working/ a été supprimé.


In [14]:
#Import image filenames
from os import listdir
from os.path import isfile, join

mypath='/kaggle/input/mycroppedimages/myCroppedImages/'
fileNames = [f for f in listdir(mypath) if isfile(join(mypath, f))]

del mypath, listdir, isfile, join

In [15]:
#Sort filenames according to frame number and store filenames in a sorted list

DfFileName=pd.DataFrame(fileNames, columns=['FileName'])
DfFileName['FrameNb']=DfFileName['FileName'].apply(lambda x: int(x.split('_')[-1].split('.')[0]))
DfFileName.sort_values(by='FrameNb', inplace=True, ignore_index=True)
#Storing sorted filenames in sortedFileNames list
sortedFileNames=[]
for _, row in DfFileName.iterrows():
    sortedFileNames.append(row['FileName'])


In [16]:
#Make copies of previous frame for missing frames
!mkdir ./missingImages
myOutPath='/kaggle/working/missingImages/'
myInpPath='/kaggle/input/mycroppedimages/myCroppedImages/'
missingFiles=[]
for img in sortedFileNames:
    if img=='cropped_frame_9965.jpg':
        imgPrev=int(img.split('_')[-1].split('.')[0])
        continue
    currImgNb=int(img.split('_')[-1].split('.')[0])
    if currImgNb==imgPrev+5:
        imgPrev=currImgNb
    else:
        iter = int((currImgNb-imgPrev)/5-1)
        newImg=cv2.imread(myInpPath+'cropped_frame_'+str(imgPrev)+'.jpg', cv2.IMREAD_COLOR)

        for i in range(1,iter+1):
            cv2.imwrite(myOutPath+'cropped_frame_'+str(imgPrev+i*5)+'.jpg', newImg)
            print('Added file ', ('cropped_frame_'+str(imgPrev+i*5)+'.jpg'))
            missingFiles.append('cropped_frame_'+str(imgPrev+i*5)+'.jpg')
        imgPrev=currImgNb
     


Added file  cropped_frame_11115.jpg
Added file  cropped_frame_11120.jpg
Added file  cropped_frame_13545.jpg
Added file  cropped_frame_13550.jpg
Added file  cropped_frame_13555.jpg
Added file  cropped_frame_13560.jpg
Added file  cropped_frame_13565.jpg
Added file  cropped_frame_13570.jpg
Added file  cropped_frame_13580.jpg
Added file  cropped_frame_13585.jpg
Added file  cropped_frame_16225.jpg
Added file  cropped_frame_16230.jpg
Added file  cropped_frame_17230.jpg
Added file  cropped_frame_17235.jpg
Added file  cropped_frame_17240.jpg
Added file  cropped_frame_19420.jpg
Added file  cropped_frame_53940.jpg
Added file  cropped_frame_53945.jpg
Added file  cropped_frame_53950.jpg
Added file  cropped_frame_114135.jpg
Added file  cropped_frame_202855.jpg
Added file  cropped_frame_202860.jpg
Added file  cropped_frame_202865.jpg
Added file  cropped_frame_202870.jpg
Added file  cropped_frame_213325.jpg


In [17]:
print(missingFiles, len(missingFiles))

['cropped_frame_11115.jpg', 'cropped_frame_11120.jpg', 'cropped_frame_13545.jpg', 'cropped_frame_13550.jpg', 'cropped_frame_13555.jpg', 'cropped_frame_13560.jpg', 'cropped_frame_13565.jpg', 'cropped_frame_13570.jpg', 'cropped_frame_13580.jpg', 'cropped_frame_13585.jpg', 'cropped_frame_16225.jpg', 'cropped_frame_16230.jpg', 'cropped_frame_17230.jpg', 'cropped_frame_17235.jpg', 'cropped_frame_17240.jpg', 'cropped_frame_19420.jpg', 'cropped_frame_53940.jpg', 'cropped_frame_53945.jpg', 'cropped_frame_53950.jpg', 'cropped_frame_114135.jpg', 'cropped_frame_202855.jpg', 'cropped_frame_202860.jpg', 'cropped_frame_202865.jpg', 'cropped_frame_202870.jpg', 'cropped_frame_213325.jpg'] 25


In [18]:
import logging
logging.getLogger("ppocr").setLevel(logging.ERROR)
from tqdm import tqdm

!mkdir ./OCRImages

myGlobalDir='/kaggle/working/OCRImages/'
myImagePath='/kaggle/input/mycroppedimages/myCroppedImages/'
#n=1    
#Prototype of function def processOneImageOCR(myImagePath, myImageName, myGlobalDir, n):

for idx, f in enumerate(tqdm(sortedFileNames, desc="Processing")):
    processOneImageOCR (myImagePath, f, myGlobalDir,1)


#Processing missing files
myMissingImagePath='/kaggle/working/missingImages/'
for idx, f in enumerate(missingFiles):
    processOneImageOCR (myMissingImagePath, f, myGlobalDir,1)

Processing:   0%|          | 1/55092 [00:02<34:49:10,  2.28s/it]

file '/kaggle/working/ocr_results_1.json' created


Processing:   3%|▎         | 1441/55092 [06:00<4:13:09,  3.53it/s]

No text detected


Processing:  16%|█▌        | 8782/55092 [38:29<1:50:04,  7.01it/s]

...

Processing:  38%|███▊      | 20817/55092 [1:22:35<1:21:25,  7.02it/s]

No text detected
No text detected


Processing:  92%|█████████▏| 50635/55092 [3:26:52<12:58,  5.72it/s]

No text detected


Processing: 100%|██████████| 55092/55092 [3:45:41<00:00,  4.07it/s]


In [19]:
#!zip -ru /kaggle/working/ocrImages2700.zip /kaggle/working/ocr_results_0.json
#!zip -ru /kaggle/working/ocrImages2700.zip /kaggle/working/ocr_results_2.json



In [20]:
import os
import shutil
from zipfile import ZipFile


base_folder = '/kaggle/working/OCRImages'
zip_file = '/kaggle/working/myImages.zip'

# Créer ou ouvrir le fichier ZIP
with ZipFile(zip_file, 'a') as zf:
    # Parcourir les sous-dossiers dans le dossier de base
    for folder_name in os.listdir(base_folder):
        folder_path = os.path.join(base_folder, folder_name)

        if os.path.isdir(folder_path):
            # Ajouter le dossier au ZIP
            for root, dirs, files in os.walk(folder_path):
                for file in files:
                    file_path = os.path.join(root, file)
                    arcname = os.path.relpath(file_path, base_folder)  # Relatif au dossier de base
                    zf.write(file_path, arcname)
            
            # Supprimer le dossier après l'avoir ajouté
            shutil.rmtree(folder_path)
            print(f"Ajouté : {folder_path}")

Ajouté : /kaggle/working/OCRImages/166820
Ajouté : /kaggle/working/OCRImages/274925
Ajouté : /kaggle/working/OCRImages/81135
Ajouté : /kaggle/working/OCRImages/137235
Ajouté : /kaggle/working/OCRImages/206290
Ajouté : /kaggle/working/OCRImages/197040
Ajouté : /kaggle/working/OCRImages/214220
Ajouté : /kaggle/working/OCRImages/191120
Ajouté : /kaggle/working/OCRImages/225780
Ajouté : /kaggle/working/OCRImages/68970
Ajouté : /kaggle/working/OCRImages/84195
Ajouté : /kaggle/working/OCRImages/48445
Ajouté : /kaggle/working/OCRImages/86840
Ajouté : /kaggle/working/OCRImages/251560
Ajouté : /kaggle/working/OCRImages/81815
Ajouté : /kaggle/working/OCRImages/120205
Ajouté : /kaggle/working/OCRImages/284195
Ajouté : /kaggle/working/OCRImages/174500
Ajouté : /kaggle/working/OCRImages/143110
Ajouté : /kaggle/working/OCRImages/125005
Ajouté : /kaggle/working/OCRImages/192345
Ajouté : /kaggle/working/OCRImages/232585
Ajouté : /kaggle/working/OCRImages/66800
Ajouté : /kaggle/working/OCRImages/16380


In [21]:
from IPython.display import FileLink

# Spécifiez le chemin du fichier à télécharger
file_path = './myImages.zip'

# Créez un lien de téléchargement
FileLink(file_path)

/kaggle/working/myImages.zip